# Receiving robot state

In this notebook, we will demonstrate how to receive and process state messages from the UR3e mockup. The mockup publishes its state messages with the `robotarm.state` routing key:
- `ROBOT_MODE`: Current mode of the robot (e.g., RUNNING, Idle).
- `Q_ACTUAL`: Current joint positions (radians).
- `QD_ACTUAL`: Current joint velocities (radians/second).
- `Q_TARGET`: Target joint positions (radians).
- `TIMESTAMP`: Timestamp of the status message.
- `JOINT_MAX_SPEED`: Maximum joint speeds (radians/second).
- `JOINT_MAX_ACCELERATION`: Maximum joint accelerations (radians/second²).
- `TCP_POSE`: Current TCP pose (position and orientation).

To receive these messages, we will set up a RabbitMQ consumer that listens for messages with the `robotarm.state` routing key. Below is an example of how to implement this in Python using the communication module provided in the project.

> Make sure the credentials match those from your RabbitMQ server setup.

In [1]:
from communication import protocol
from communication.rabbitmq import Rabbitmq

try:
    rmq = Rabbitmq(
        ip="localhost",
        port=5672,
        username="ur3e",
        password="ur3e",
        vhost="/",
        exchange="UR3E_AMQP",
        type="topic",
    )
    rmq.connect_to_server()
    print("✓ Connected to RabbitMQ successfully")
except Exception as e:
    print(f"✗ Failed to connect to RabbitMQ: {e}")
    print("\nMake sure RabbitMQ is running. You can start it with:")
    print("  python -m startup.start_docker_rabbitmq")

# define a callback function to handle received messages
def on_message_received(ch, method, properties, body):
    try:
        print("✓ State:")
        print(body)
    except Exception as e:
        print(f"✗ Failed to decode the message: {e}")

✓ Connected to RabbitMQ successfully


Running the next cell will set up the subscription and start receiving state messages from the mockup. Note that this will block the notebook until interrupted, which is in

In [2]:
rmq.subscribe(
    routing_key=protocol.ROUTING_KEY_STATE,
    on_message_callback=on_message_received,
)

rmq.start_consuming()

✓ State:
{'robot_mode': 'Idle', 'q_actual': [-1.7005687636691133e-07, -1.5707936010143788, 1.5707942159291972, -1.5707975178483689, -1.5707978858368585, -1.941640305418051e-06], 'qd_actual': [0.0, 0.0, 0.0, 0.0, 0.0, -5.421010862427522e-20], 'q_target': [0.0, -1.5707963267948966, 1.5707963267948966, -1.5707963267948966, -1.5707963267948966, -3.3881317890172014e-21], 'timestamp': 310.60000000003635, 'joint_max_speed': 60, 'joint_max_acceleration': 80, 'tcp_pose': [-0.2985360517301431, -0.13105203884893465, 0.3033055324374535, -1.5707980983774275, -5.761383887695376e-07, -3.141591094547733]}
✓ State:
{'robot_mode': 'Idle', 'q_actual': [-1.626413205514563e-07, -1.5707936791171173, 1.5707942817757763, -1.5707974715470352, -1.5707978774508597, -1.968706354640817e-06], 'qd_actual': [0.0, 0.0, 0.0, 0.0, 0.0, -5.421010862427522e-20], 'q_target': [0.0, -1.5707963267948966, 1.5707963267948966, -1.5707963267948966, -1.5707963267948966, -3.3881317890172014e-21], 'timestamp': 310.65000000003636, 'j

KeyboardInterrupt: 

Try to modify the callback function to log the received state messages in a more structured format, such as a csv file. 

In [3]:
rmq.close()